## Запрос №3

Напишите запрос, выбирающий информацию по всем служащим,
нанятым раньше своих менеджеров. При этом выведите фамилии и даты
найма самих служащих, а также фамилии и даты найма их менеджеров.
Пример возможного результата выполнения запроса представлен в
таблице ниже. Поля должны называться так же, как указано в примере.


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format


spark = SparkSession.builder.master("local").getOrCreate()
fmt = "dd.MM.yyyy"
empDf = spark.read.parquet("data/employees.parquet").withColumn(
    "hire_date", to_date(col("hire_date"), fmt)
)
# empDf.printSchema()

df = (
    empDf.select("manager_id", "last_name", "hire_date")
    .withColumnRenamed("last_name", "Фамилия_Р")
    .withColumnRenamed("hire_date", "date_w")
)
df2 = empDf.select("employee_id", "last_name", "hire_date")
(
    df.join(df2, df.manager_id == df2.employee_id)
    .orderBy(col("hire_date"))
    .filter(col("hire_date") > col("date_w"))
    .select(
        "Фамилия_Р",
        date_format(col("date_w"), fmt).alias("Дата_Р"),
        col("last_name").alias("Фамилия_М"),
        date_format(col("hire_date"), fmt).alias("Дата_М"),
    )
).show()

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- job_id: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- commission_pct: string (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)

+---------+----------+---------+----------+
|Фамилия_Р|    Дата_Р|Фамилия_М|    Дата_М|
+---------+----------+---------+----------+
|   Faviet|16.08.2002|Greenberg|17.08.2002|
| Raphaely|07.12.2002|     King|17.06.2003|
|  De Haan|13.01.2001|     King|17.06.2003|
| Kaufling|01.05.2003|     King|17.06.2003|
|    Sully|04.03.2004| Partners|05.01.2005|
|   McEwen|01.08.2004| Partners|05.01.2005|
|     King|30.01.2004| Partners|05.01.2005|
|   Marlow|16.02.2005|    Fripp|10.04.2005|
|     Bull|20.02.2005|    Fripp|10.04.2005|
| Sarchan